# FnGuide Crawling

In [2]:
#!/usr/bin/env python3
"""
리팩토링된 동적 테이블 크롤러
- 함수 분리로 가독성 향상
- 클래스 기반 구조화
- 에러 처리 개선
- 디버그 모드 분리
"""

import requests
import pandas as pd
from bs4 import BeautifulSoup, Tag
from typing import Dict, List, Tuple, Optional
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [6]:
# 파티션 폴더명 접두사 (GCS 저장 경로용)
QUARTER_PREFIX = "quarter="

# 정적 HTML에서 가져올 테이블 (원본 utils/fnguide.py)
STATIC_TABLE_MAP = [
    ("market_conditions", 0),
    ("earning_issue", 1),
    ("holdings_status", 2),
    ("governance", 3),
    ("shareholders", 4),
    ("bond_rating", 6),
    ("analysis", 7),
    ("industry_comparison", 8),
    ("financialhighlight_annual", 11),
    ("financialhighlight_netquarter", 12),
]

DYNAMIC_TABLE_TITLES = ["포괄손익계산서", "재무상태표", "현금흐름표"]

In [9]:
static_url = "https://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A{stock}"
dynamic_url = "https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{stock}&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701"

## 재무제표 정보

In [3]:
# =============================================================================
# 헬퍼 클래스들
# =============================================================================

class TableFinder:
    """테이블 찾기 전담 클래스"""
    
    @staticmethod
    def find_by_title(soup: BeautifulSoup, title: str) -> Optional[Tag]:
        """제목으로 테이블 찾기"""
        for table in soup.find_all("table"):
            if title in table.get_text():
                logger.info(f"'{title}' 테이블 발견")
                return table
        
        logger.warning(f"'{title}' 테이블을 찾을 수 없음")
        return None


class HeaderExtractor:
    """테이블 헤더 추출 전담 클래스"""
    
    @staticmethod
    def extract_index_list(thead: Tag) -> List[str]:
        """
        thead에서 인덱스 리스트 추출
        
        Args:
            thead: thead 태그
        
        Returns:
            인덱스 리스트 (날짜/기간 데이터)
        """
        thead_rows = thead.find_all("tr")
        index_rows: List[List[str]] = []
        
        for tr in thead_rows:
            row_headers = HeaderExtractor._extract_row_headers(tr)
            if row_headers:
                index_rows.append(row_headers)
        
        # 가장 긴 행을 선택 (가장 상세한 헤더)
        index_list = max(index_rows, key=len) if index_rows else []
        
        total_headers = sum(len(tr.find_all("th")) for tr in thead_rows)
        logger.info(f"기간 데이터 (헤더 {total_headers}개 중 {len(index_list)}개): {index_list}")
        
        return index_list
    
    @staticmethod
    def _extract_row_headers(tr: Tag) -> List[str]:
        """
        단일 행에서 헤더 추출 (colspan 처리)
        
        Args:
            tr: tr 태그
        
        Returns:
            헤더 리스트
        """
        row_headers = []
        ths = tr.find_all("th", recursive=False)
        
        for col_idx, th in enumerate(ths):
            text = th.get_text(strip=True)
            if not text:
                continue
            
            # 첫 번째 th는 행 구분자로 스킵 (2개 이상일 때만)
            if col_idx == 0 and len(ths) > 1:
                continue
            
            # colspan 처리
            colspan = HeaderExtractor._get_colspan(th)
            row_headers.extend([text] * colspan)
        
        return row_headers
    
    @staticmethod
    def _get_colspan(th: Tag) -> int:
        """colspan 속성 추출"""
        colspan_attr = th.get("colspan")
        try:
            return int(colspan_attr) if colspan_attr else 1
        except ValueError:
            return 1


class BodyExtractor:
    """테이블 바디 추출 전담 클래스"""
    
    def __init__(self, debug: bool = False):
        self.debug = debug
        self.last_span_text: Optional[str] = None
    
    def extract(
        self,
        tbody: Tag,
        index_list: List[str]
    ) -> Dict[Tuple[str, str], List[str]]:
        """
        tbody에서 데이터 딕셔너리 추출
        
        Args:
            tbody: tbody 태그
            index_list: 헤더 인덱스 리스트
        
        Returns:
            {(카테고리, 항목): [값들]} 형태의 딕셔너리
        """
        data_dict = {}
        tbody_trs = tbody.find_all("tr", recursive=False)
        
        logger.info(f"발견된 행 수: {len(tbody_trs)}")
        
        if self.debug and tbody_trs:
            logger.debug(f"첫 행 HTML: {str(tbody_trs[0])[:500]}")
        
        processed_count = 0
        
        for idx, tr in enumerate(tbody_trs):
            self._log_progress(idx, len(tbody_trs))
            
            try:
                # 행 데이터 추출
                column_tuple, values = self._extract_row_data(tr, idx)
                
                if not column_tuple or not values:
                    continue
                
                # 헤더와 값 개수 검증
                if len(values) == len(index_list):
                    data_dict[column_tuple] = values
                    processed_count += 1
                    
                    if self.debug and idx < 3:
                        logger.debug(f"행 {idx} 성공!")
                else:
                    if self.debug and idx < 5:
                        logger.warning(
                            f"행 {idx} 길이 불일치: "
                            f"헤더={len(index_list)}, 값={len(values)} (스킵)"
                        )
            
            except Exception as e:
                if self.debug and idx < 5:
                    logger.error(f"행 {idx} 처리 중 에러: {e}")
                continue
        
        logger.info(f"처리 완료: {processed_count}/{len(tbody_trs)} 행")
        
        return data_dict
    
    def _extract_row_data(
        self,
        tr: Tag,
        idx: int
    ) -> Tuple[Optional[Tuple[str, str]], List[str]]:
        """
        단일 행에서 컬럼명과 값 추출
        
        Args:
            tr: tr 태그
            idx: 행 인덱스
        
        Returns:
            (컬럼명_튜플, 값_리스트)
        """
        # th 찾기
        th = tr.find("th")
        if not th:
            if self.debug and idx < 3:
                logger.debug(f"행 {idx}: th 없음")
            return None, []
        
        # 컬럼명 추출
        column_tuple = self._extract_column_name(th, idx)
        
        # td 값들 추출
        tds = tr.find_all("td", recursive=False)
        values = [td.get_text(strip=True) for td in tds]
        
        if self.debug and idx < 3:
            logger.debug(
                f"행 {idx}: column={column_tuple}, "
                f"td 개수={len(tds)}, 값={values[:3] if values else '없음'}"
            )
        
        return column_tuple, values
    
    def _extract_column_name(self, th: Tag, idx: int) -> Tuple[str, str]:
        """
        th에서 컬럼명 튜플 추출 (span 고려)
        
        Args:
            th: th 태그
            idx: 행 인덱스
        
        Returns:
            (카테고리, 항목) 튜플
        """
        span = th.find("span")
        th_text = th.get_text(strip=True)
        
        # span이 있는 경우: 새로운 상위 카테고리
        if span:
            span_text = span.get_text(strip=True)
            self.last_span_text = span_text
            column_tuple = (span_text, th_text)
            
            if self.debug and idx < 3:
                logger.debug(f"행 {idx} (span): {column_tuple}")
        
        # span 없는 경우: 이전 카테고리 사용
        else:
            if self.last_span_text:
                column_tuple = (self.last_span_text, th_text)
            else:
                column_tuple = (th_text, "")
            
            if self.debug and idx < 3:
                logger.debug(f"행 {idx} (no span): {column_tuple}")
        
        return column_tuple
    
    @staticmethod
    def _log_progress(idx: int, total: int):
        """진행상황 로깅 (20개마다)"""
        if idx > 0 and idx % 20 == 0:
            logger.info(f"처리 중: {idx}/{total} 행")

In [12]:
# =============================================================================
# 메인 파서 클래스
# =============================================================================

class FinancialTableParser:
    """재무제표 테이블 파서"""
    
    def __init__(self, debug: bool = False):
        """
        Args:
            debug: 디버그 모드
        """
        self.debug = debug
        self.table_finder = TableFinder()
        self.header_extractor = HeaderExtractor()
    
    def parse_table(
        self,
        soup: BeautifulSoup,
        title: str
    ) -> Optional[pd.DataFrame]:
        """
        단일 테이블 파싱
        
        Args:
            soup: BeautifulSoup 객체
            title: 테이블 제목
        
        Returns:
            DataFrame 또는 None
        """
        logger.info(f"\n{title} 데이터 수집 중...")
        
        # 1. 테이블 찾기
        table = self.table_finder.find_by_title(soup, title)
        if not table:
            return None
        
        # 2. thead 추출
        thead = table.find("thead")
        if not thead:
            logger.warning("thead를 찾을 수 없음")
            return None
        
        # 3. 헤더 인덱스 추출
        index_list = self.header_extractor.extract_index_list(thead)
        if not index_list:
            logger.warning("인덱스 리스트가 비어있음")
            return None
        
        # 4. tbody 추출
        tbody = table.find("tbody")
        if not tbody:
            logger.warning("tbody를 찾을 수 없음")
            return None
        
        # 5. 데이터 딕셔너리 추출
        body_extractor = BodyExtractor(debug=self.debug)
        data_dict = body_extractor.extract(tbody, index_list)
        
        if not data_dict:
            logger.warning("데이터가 비어있음")
            return None
        
        # 6. DataFrame 생성
        df = self._create_dataframe(data_dict, index_list)
        
        logger.info(f"완료! DataFrame shape: {df.shape}")
        
        return df
    
    @staticmethod
    def _create_dataframe(
        data_dict: Dict[Tuple[str, str], List[str]],
        index_list: List[str]
    ) -> pd.DataFrame:
        """
        데이터 딕셔너리에서 DataFrame 생성
        
        Args:
            data_dict: {(카테고리, 항목): [값들]} 딕셔너리
            index_list: 인덱스 리스트
        
        Returns:
            멀티인덱스 컬럼을 가진 DataFrame
        """
        df = pd.DataFrame(data_dict, index=index_list)
        
        # 멀티인덱스로 컬럼 변환
        df.columns = pd.MultiIndex.from_tuples(df.columns)
        
        return df

In [5]:
# =============================================================================
# 메인 함수 (리팩토링 버전)
# =============================================================================

def get_dynamic_tables(
    stock: str,
    dynamic_url: str,
    table_titles: List[str],
    debug: bool = False
) -> Dict[str, List[Dict]]:
    """
    동적 테이블 수집
    
    Args:
        stock: 종목 코드
        dynamic_url: URL 템플릿 ('{stock}' 포함)
        table_titles: 수집할 테이블 제목 리스트
        debug: 디버그 모드
    
    Returns:
        {테이블명: 레코드_리스트} 딕셔너리
    """
    result_dict = {}
    
    # 1. 페이지 가져오기
    url = dynamic_url.format(stock=stock)
    logger.info(f"Dynamic URL: {url}")
    
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
    except requests.RequestException as e:
        logger.error(f"페이지 요청 실패: {e}")
        return {title: [] for title in table_titles}
    
    # 2. BeautifulSoup 파싱
    soup = BeautifulSoup(response.text, "html.parser")
    
    # 3. 파서 생성
    parser = FinancialTableParser(debug=debug)
    
    # 4. 각 테이블 파싱
    for title in table_titles:
        try:
            df = parser.parse_table(soup, title)
            
            if df is not None:
                # DataFrame을 레코드로 변환
                result_dict[title] = dataframe_to_records(df)
            else:
                result_dict[title] = []
        
        except Exception as e:
            logger.error(f"{title} 수집 실패: {e}")
            result_dict[title] = []
    
    return result_dict


def dataframe_to_records(df: pd.DataFrame) -> List[Dict]:
    """
    DataFrame을 레코드 리스트로 변환
    
    Args:
        df: 변환할 DataFrame
    
    Returns:
        레코드 딕셔너리 리스트
    """
    return df.to_dict('records')

In [13]:
# 실행
result = get_dynamic_tables(
    stock="005930",
    dynamic_url=dynamic_url,
    table_titles=DYNAMIC_TABLE_TITLES,
    debug=True  # 디버그 모드
)

INFO:__main__:Dynamic URL: https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A005930&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701
INFO:__main__:
포괄손익계산서 데이터 수집 중...
INFO:__main__:'포괄손익계산서' 테이블 발견
INFO:__main__:기간 데이터 (헤더 7개 중 6개): ['2022/12', '2023/12', '2024/12', '2025/09', '전년동기', '전년동기(%)']
INFO:__main__:발견된 행 수: 79
INFO:__main__:처리 중: 20/79 행
INFO:__main__:처리 중: 40/79 행
INFO:__main__:처리 중: 60/79 행
INFO:__main__:처리 완료: 79/79 행
INFO:__main__:완료! DataFrame shape: (6, 79)
INFO:__main__:
재무상태표 데이터 수집 중...
INFO:__main__:'재무상태표' 테이블 발견
INFO:__main__:기간 데이터 (헤더 5개 중 4개): ['2022/12', '2023/12', '2024/12', '2025/09']
INFO:__main__:발견된 행 수: 66
INFO:__main__:처리 중: 20/66 행
INFO:__main__:처리 중: 40/66 행
INFO:__main__:처리 중: 60/66 행
INFO:__main__:처리 완료: 66/66 행
INFO:__main__:완료! DataFrame shape: (4, 66)
INFO:__main__:
현금흐름표 데이터 수집 중...
INFO:__main__:'현금흐름표' 테이블 발견
INFO:__main__:기간 데이터 (헤더 5개 중 4개): ['2022/12', '2023/12', '2024/12', '2025/09']
INFO:__main__:발견된 행 수: 158
INFO:__main__:

In [14]:
result

{'포괄손익계산서': [{('매출액', ''): '3,022,314',
   ('매출원가', ''): '1,900,418',
   ('매출총이익', ''): '1,121,896',
   ('판매비와관리비', '판매비와관리비계산에 참여한 계정 펼치기'): '688,130',
   ('판매비와관리비', '인건비'): '80,937',
   ('판매비와관리비', '유무형자산상각비'): '22,391',
   ('판매비와관리비', '연구개발비'): '249,192',
   ('판매비와관리비', '광고선전비'): '61,130',
   ('판매비와관리비', '판매비'): '139,969',
   ('판매비와관리비', '관리비'): '74,579',
   ('판매비와관리비', '기타원가성비용'): '',
   ('판매비와관리비', '기타'): '59,932',
   ('판매비와관리비', '영업이익'): '433,766',
   ('판매비와관리비', '영업이익(발표기준)'): '433,766',
   ('금융수익', '금융수익계산에 참여한 계정 펼치기'): '208,290',
   ('금융수익', '이자수익'): '27,205',
   ('금융수익', '배당금수익'): '',
   ('금융수익', '외환이익'): '165,379',
   ('금융수익', '대손충당금환입액'): '',
   ('금융수익', '매출채권처분이익'): '',
   ('금융수익', '당기손익-공정가치측정\xa0금융자산관련이익'): '',
   ('금융수익', '금융자산처분이익'): '',
   ('금융수익', '금융자산평가이익'): '',
   ('금융수익', '금융자산손상차손환입'): '',
   ('금융수익', '파생상품이익'): '15,707',
   ('금융수익', '기타금융수익'): '',
   ('금융원가', '금융원가계산에 참여한 계정 펼치기'): '190,277',
   ('금융원가', '이자비용'): '7,630',
   ('금융원가', '외환손실'): '168,097',
   ('

## Snapshot | 기업정보 | Company Guide